In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

24/06/10 18:22:43 WARN Utils: Your hostname, Arbnors-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.11.6.58 instead (on interface en0)
24/06/10 18:22:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/10 18:22:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/10 18:22:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/10 18:22:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [22]:
myCollection = "This is my Name and Surname".split(" ")

In [23]:
words = spark.sparkContext.parallelize(myCollection, 2)

In [24]:
supplementData = {"Name": 100, "Surname": 200,
                  "Stamina": -300, "Fit": 400}

In [25]:
suppBroadcast = spark.sparkContext.broadcast(supplementData)

In [26]:
suppBroadcast.value

{'Name': 100, 'Surname': 200, 'Stamina': -300, 'Fit': 400}

In [27]:
value = suppBroadcast.value

In [28]:


words.map(lambda word: (word, suppBroadcast.value.get(word, 0)))\
    .sortBy(lambda matchWord: matchWord[1])\
    .collect()

[('This', 0),
 ('is', 0),
 ('my', 0),
 ('and', 0),
 ('Name', 100),
 ('Surname', 200)]

In [30]:
flights = spark.read.parquet("data/flight-data/parquet/2010-summary.parquet")

In [32]:
flights.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=264),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=69),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=24),
 Row(DEST_COUNTRY_NAME='Equatorial Guinea', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [45]:
flights.select("*").where("DEST_COUNTRY_NAME == 'United States'").show(10, False)

+-----------------+------------------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME           |count|
+-----------------+------------------------------+-----+
|United States    |Romania                       |1    |
|United States    |Ireland                       |264  |
|United States    |India                         |69   |
|United States    |Singapore                     |25   |
|United States    |Grenada                       |54   |
|United States    |Marshall Islands              |44   |
|United States    |Sint Maarten                  |53   |
|United States    |Afghanistan                   |2    |
|United States    |Russia                        |156  |
|United States    |Federated States of Micronesia|48   |
+-----------------+------------------------------+-----+
only showing top 10 rows



In [46]:
accChina = spark.sparkContext.accumulator(0)

In [47]:
def accChinaFunc(flightRow):
    destination = flightRow['DEST_COUNTRY_NAME']
    origin = flightRow['ORIGIN_COUNTRY_NAME']

    if destination == 'China':
        accChina.add(flightRow['count'])

    if origin == 'China':
        accChina.add(flightRow['count'])

In [56]:
flights.foreach(lambda flightrow: accChinaFunc(flightrow))

In [58]:
accChina.value

3812